# Import Library

In [1]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import sys
import os
import ast

code_path = os.path.abspath('../')
sys.path.append(code_path)

from credentials import connection_string

# Import Data

## Connect to MongoDB Cluster

In [2]:
database_name = "Singaraja"
collection_name = f"{database_name}_Raw"

In [3]:
client = MongoClient(connection_string)
db = client[database_name]
collection = db[collection_name]

## Get All Documents from Selected Collection

In [4]:
all_documents = collection.find()

## Convert to DataFrame

In [5]:
df = pd.DataFrame(list(all_documents))

# drop _id
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [6]:
df

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']","['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made...",3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,117/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",49 Hari,"['MADE SUDAMA ALIAS KARTOLO', 'PUTU WISNU ALIA...","['I MADE SUTAPA,S.H.']","['Made Hermayanti Muliartha~Hakim Ketua~Ya', '...",3,['PUTU WISNU ALIAS WISNU~Subsider Penjara (10 ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudama ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,112/Pid.Sus/2024/PN Sgr,Narkotika,"Selasa, 30 Jul. 2024",44 Hari,"['I KOMANG TEKEN Alias TEKEN', 'KADEK ALBET SA...","['Kadek Adi Pramarta, S.H.']","['I Made Bagiarta~Hakim Ketua~Tidak', 'I Gusti...",4,['KADEK ALBET SANJAYA Alias ALBET~Pidana Penja...,M E N G A D I L I :\nMenyatakan terda...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...
3,Minutasi,101/Pid.B/2024/PN Sgr,Pencurian,"Kamis, 11 Jul. 2024",56 Hari,['ANDRIANI'],"['Nyoman Arif Budiman, S.H., M.H.']","['I Gusti Made Juliartawan~Hakim Ketua~Ya', 'N...",3,['ANDRIANI~Pidana Penjara Waktu Tertentu (1 Ta...,"MENGADILI:\nMenyatakan Terdakwa ANDRIANI, tela...",“Demi Keadilan dan Kebenaran\nBerdasarkan Ketu...
4,Minutasi,98/Pid.Sus/2024/PN Sgr,Narkotika,"Rabu, 10 Jul. 2024",64 Hari,['I NYOMAN IWAN MAHADI alias MANG IWAN'],"['I GUSTI NGURAH ARYA DIATMIKA,S.H.']","['I Made Bagiarta~Hakim Ketua~Ya', 'Made Herma...",3,['I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana ...,MENGADILI:\nMenyatakan Terdakwa I Nyoman Iwan ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
...,...,...,...,...,...,...,...,...,...,...,...,...
1265,Minutasi,237/Pid.B/2017/PN Sgr,Penggelapan,"Rabu, 20 Des. 2017",56 Hari,[Ayu Putu Erlyandani],"[IMAM EKA SETYAWAN, SH.]",[Ida Ayu Sri Adriyanthi Astuti Widja~Hakim Ket...,4,[Ayu Putu Erlyandani~Pidana Penjara Waktu Tert...,M E N G A D I L I:\nMenyatakan Terdakwa AYU PU...,Kesatu : Â \nPrimair.\n-------- Bahwa terdakwa...
1266,Minutasi,236/Pid.B/2017/PN Sgr,Kejahatan terhadap Nyawa,"Rabu, 20 Des. 2017",97 Hari,[Ketut Mahardika Alias Kelet],"[I Nyoman Sulitra, SH.,MH.]",[Ida Ayu Sri Adriyanthi Astuti Widja~Hakim Ket...,10,[Ketut Mahardika Alias Kelet~Pidana Penjara Wa...,MENGADILI:\nÂ \nMenyatakan Terdakwa Ketut Maha...,Kesatu\n----- Bahwa terdakwa Â KETUT MAHARDIKA...
1267,Minutasi,227/Pid.B/2017/PN Sgr,Penggelapan,"Senin, 11 Des. 2017",52 Hari,[Iwan Hermato Alias Iwan],"[I GEDE PUTU ASTAWA, SH.]","[Ida Bagus Bama Dewa. P~Hakim Ketua~Ya, I Made...",3,[Iwan Hermato Alias Iwan~Pidana Kurungan (1 Ta...,MENGADILI\nMenyatakan Terdakwa Iwan Hermanto A...,Pertama : Pasal 372 Kitab Undang-undang Hukum ...
1268,Minutasi,226/Pid.Sus/2017/PN Sgr,Narkotika,"Senin, 11 Des. 2017",31 Hari,[Komang Irwin Pranata Alias Erwin],"[I GEDE PUTU ASTAWA, SH.]","[Sudar~Hakim Ketua~Ya, Anak Agung Ngurah Budhi...",2,[Komang Irwin Pranata Alias Erwin~Pidana Penja...,Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...,Pertama : Pasal 112 ayat (1) UU RI No. 35 tahu...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   status_perkara       1270 non-null   object
 1   nomor_perkara        1270 non-null   object
 2   klasifikasi_perkara  1270 non-null   object
 3   tanggal_pendaftaran  1270 non-null   object
 4   lama_proses          1270 non-null   object
 5   terdakwa             1270 non-null   object
 6   penuntut_umum        1270 non-null   object
 7   hakim                1270 non-null   object
 8   jumlah_saksi         1270 non-null   int64 
 9   putusan_hukuman      1270 non-null   object
 10  barang_bukti         1270 non-null   object
 11  dakwaan              1267 non-null   object
dtypes: int64(1), object(11)
memory usage: 119.2+ KB


# Data Understanding

In [8]:
df.head(5)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']","['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made...",3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,117/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",49 Hari,"['MADE SUDAMA ALIAS KARTOLO', 'PUTU WISNU ALIA...","['I MADE SUTAPA,S.H.']","['Made Hermayanti Muliartha~Hakim Ketua~Ya', '...",3,['PUTU WISNU ALIAS WISNU~Subsider Penjara (10 ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudama ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,112/Pid.Sus/2024/PN Sgr,Narkotika,"Selasa, 30 Jul. 2024",44 Hari,"['I KOMANG TEKEN Alias TEKEN', 'KADEK ALBET SA...","['Kadek Adi Pramarta, S.H.']","['I Made Bagiarta~Hakim Ketua~Tidak', 'I Gusti...",4,['KADEK ALBET SANJAYA Alias ALBET~Pidana Penja...,M E N G A D I L I :\nMenyatakan terda...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...
3,Minutasi,101/Pid.B/2024/PN Sgr,Pencurian,"Kamis, 11 Jul. 2024",56 Hari,['ANDRIANI'],"['Nyoman Arif Budiman, S.H., M.H.']","['I Gusti Made Juliartawan~Hakim Ketua~Ya', 'N...",3,['ANDRIANI~Pidana Penjara Waktu Tertentu (1 Ta...,"MENGADILI:\nMenyatakan Terdakwa ANDRIANI, tela...",“Demi Keadilan dan Kebenaran\nBerdasarkan Ketu...
4,Minutasi,98/Pid.Sus/2024/PN Sgr,Narkotika,"Rabu, 10 Jul. 2024",64 Hari,['I NYOMAN IWAN MAHADI alias MANG IWAN'],"['I GUSTI NGURAH ARYA DIATMIKA,S.H.']","['I Made Bagiarta~Hakim Ketua~Ya', 'Made Herma...",3,['I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana ...,MENGADILI:\nMenyatakan Terdakwa I Nyoman Iwan ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...


**Dataset Features**
| Fitur | Deskripsi | Tipe Data |
|-----------------------|-----------------------------------------------------------------------|:------------:|
| `status_perkara` | Status dari perkara.| `str` |
| `nomor_perkara` | Nomor identifikasi unik untuk perkara.| `str` |
| `klasifikasi_perkara` | Kategori atau jenis perkara.| `str` |
| `tanggal_pendaftaran` | Tanggal ketika perkara didaftarkan di pengadilan.| `str` |
| `lama_proses` | Durasi waktu yang dibutuhkan untuk memproses perkara.| `str` |
| `terdakwa` | Nama individu yang didakwa dalam perkara tersebut.| `list[str]` |
| `penuntut_umum` | Nama jaksa atau penuntut yang menangani perkara.| `list[str]`  |
| `hakim` | Nama hakim yang memimpin sidang perkara.| `list[str]` |
| `jumlah_saksi` | Total jumlah saksi yang dihadirkan dalam sidang.| `int` |
| `putusan_hukuman` | Putusan atau keputusan yang dijatuhkan oleh hakim.| `list[str]` |
| `barang_bukti` | Amar Putusan yang berisi tentang barang bukti perkara.| `str` |
| `dakwaan`| Uraian mengenai tuduhan atau dakwaan yang diajukan terhadap terdakwa.| `str` |







# Data Preprocessing

## Data Cleaning

### Data Cleaning: Null Values

In [9]:
# Check for null values in the entire DataFrame
null_counts = df.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                3
dtype: int64


In [10]:
df_copy = df.dropna()

In [11]:
df_copy = df.dropna()
# Check for null values in the entire DataFrame
null_counts = df_copy.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


### Data Cleaning: Data Disamarkan

In [12]:
count_disamarkan = df_copy[df_copy["dakwaan"] == "Disamarkan"].shape[0]
print(f"Jumlah data 'Disamarkan': {count_disamarkan}")

Jumlah data 'Disamarkan': 100


In [13]:
df_copy = df_copy[df_copy["dakwaan"] != "Disamarkan"]
df_copy.reset_index(drop=True, inplace=True)

### Data Cleaning: Status Perkara

In [14]:
unique_status_perkara_values = df_copy['status_perkara'].unique()
num_unique_status_perkara = len(unique_status_perkara_values)

print(f"Number of unique values in 'status_perkara': {num_unique_status_perkara}")
print("Unique values:", unique_status_perkara_values)


Number of unique values in 'status_perkara': 1
Unique values: ['Minutasi']


### Data Cleaning: Nomor Perkara

In [15]:
unique_nomor_perkara_values = df_copy['nomor_perkara'].unique()
num_unique_nomor_perkara = len(unique_nomor_perkara_values)

print(f"Jumlah Data: {len(df_copy)}")
print(f"Jumlah Nilai Unique pada Kolom 'nomor_perkara': {num_unique_nomor_perkara}")

Jumlah Data: 1167
Jumlah Nilai Unique pada Kolom 'nomor_perkara': 1167


### Data Cleaning: Klasifikasi Perkara

In [16]:
unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
print(unique_klasifikasi_counts)

Number of unique values in 'status_perkara': 36
klasifikasi_perkara
Pencurian                                                                                               332
Narkotika                                                                                               323
Penganiayaan                                                                                            104
Penggelapan                                                                                              88
Lalu Lintas                                                                                              76
Kejahatan Perjudian                                                                                      63
Lain-Lain                                                                                                55
Penipuan                                                                                                 28
Pembunuhan                                                          

### Data Cleaning: Hakim

In [17]:
# Count unique data types in the column
unique_types = df_copy['hakim'].apply(type).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'> <class 'list'>]


In [18]:
for index, row in df_copy.iterrows():
    value = row['hakim']
    print(f"Type: {type(value)}, Row {index} - Value: {value}")

Type: <class 'str'>, Row 0 - Value: ['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']
Type: <class 'str'>, Row 1 - Value: ['Made Hermayanti Muliartha~Hakim Ketua~Ya', 'Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya', 'Wayan Eka Satria Utama~Hakim Anggota~Ya']
Type: <class 'str'>, Row 2 - Value: ['I Made Bagiarta~Hakim Ketua~Tidak', 'I Gusti Made Juliartawan~Hakim Anggota~Tidak', 'Ni Putu Asih Yudiastri~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'I Gusti Made Juliartawan~Hakim Anggota~Ya', 'I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya']
Type: <class 'str'>, Row 3 - Value: ['I Gusti Made Juliartawan~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya']
Type: <class 'str'>, Row 4 - Value: ['I Made Bagiarta~Hakim Ketua~Ya', 'Made Hermay

In [19]:
for index, row in df_copy.iterrows():
    value = row["hakim"]
    
    # If the value is a string representation of a list, convert it back to a list
    if isinstance(value, str):
        try:
            value = ast.literal_eval(value)  # Convert the string back to a list
            df_copy.at[index, "hakim"] = value  # Update the value in the DataFrame
        except (ValueError, SyntaxError):
            print(f"Error in Row {index}: Not a valid list format.")
    
    print(f"Type: {type(value)}, Row {index} - Value: {value}")

Type: <class 'list'>, Row 0 - Value: ['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']
Type: <class 'list'>, Row 1 - Value: ['Made Hermayanti Muliartha~Hakim Ketua~Ya', 'Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya', 'Wayan Eka Satria Utama~Hakim Anggota~Ya']
Type: <class 'list'>, Row 2 - Value: ['I Made Bagiarta~Hakim Ketua~Tidak', 'I Gusti Made Juliartawan~Hakim Anggota~Tidak', 'Ni Putu Asih Yudiastri~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'I Gusti Made Juliartawan~Hakim Anggota~Ya', 'I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya']
Type: <class 'list'>, Row 3 - Value: ['I Gusti Made Juliartawan~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya']
Type: <class 'list'>, Row 4 - Value: ['I Made Bagiarta~Hakim Ketua~Ya', 'Made H

In [20]:
# Count unique data types in the column
unique_types = df_copy['hakim'].apply(type).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


In [21]:
df_copy["hakim"][0]

['I Made Bagiarta~Hakim Ketua~Tidak',
 'Ni Made Kushandari~Hakim Anggota~Tidak',
 'Made Astina Dwipayana~Hakim Anggota~Tidak',
 'I Made Bagiarta~Hakim Ketua~Ya',
 'Ni Made Kushandari~Hakim Anggota~Ya',
 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']

In [22]:
hakim_per_case_list =[]
for index, row in df_copy.iterrows():
    hakim_per_case = len(row["hakim"])

    hakim_per_case_list.append(hakim_per_case)

unique_hakim_per_case_set = set(hakim_per_case_list)
print(f"Unique values : {unique_hakim_per_case_set}")

Unique values : {9, 3, 6}


In [23]:
indices_with_nine = []

for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 9: 
        print(row["hakim"])
        print(row["nomor_perkara"])
        print(index)

['I Gede Yuliartha~Hakim Ketua~Tidak', 'A.A. Ayu Merta Dewi~Hakim Anggota~Tidak', 'Anak Agung Ngurah Budhi Dharmawan~Hakim Anggota~Tidak', 'I Gede Yuliartha~Hakim Ketua~Tidak', 'Anak Agung Ngurah Budhi Dharmawan~Hakim Anggota~Tidak', 'Wayan Eka Satria Utama~Hakim Anggota~Tidak', 'I Gede Yuliartha~Hakim Ketua~Ya', 'Anak Agung Ngurah Budhi Dharmawan~Hakim Anggota~Ya', 'I Nyoman Dipa Rudiana~Hakim Anggota~Ya']
184/Pid.Sus/2020/PN Sgr
493


In [24]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

I Made Bagiarta~Hakim Ketua~Tidak
Ni Made Kushandari~Hakim Anggota~Tidak
Made Astina Dwipayana~Hakim Anggota~Tidak
I Made Bagiarta~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

Made Hermayanti Muliartha~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Wayan Eka Satria Utama~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Tidak
I Gusti Made Juliartawan~Hakim Anggota~Tidak
Ni Putu Asih Yudiastri~Hakim Anggota~Tidak
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Anggota~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
Made Hermayanti Muliartha~Hakim Anggota~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Pulung Yustisia Dewi~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gust

In [25]:
def remove_hakim_tidak_aktif(list_of_entries):
    return [entry for entry in list_of_entries if '~Tidak' not in entry]

In [26]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(remove_hakim_tidak_aktif)

In [27]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

I Made Bagiarta~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

Made Hermayanti Muliartha~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Wayan Eka Satria Utama~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Anggota~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
Made Hermayanti Muliartha~Hakim Anggota~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Pulung Yustisia Dewi~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya
Made Astina Dwipayana~Hakim Anggota~Ya

I Gusti Ayu Kade Ari Wulandari~Hakim Ketua~Ya
Made Astina Dwipayana~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Y

In [28]:
cleaned_hakim_per_case_list =[]
for index, row in df_copy.iterrows():
    cleaned_hakim_per_case = len(row["hakim"])

    cleaned_hakim_per_case_list.append(hakim_per_case)

unique_cleaned_hakim_per_case_set = set(cleaned_hakim_per_case_list)
print(f"Unique values : {unique_cleaned_hakim_per_case_set}")

Unique values : {3}


In [29]:
for index, row in df_copy.iterrows():
    df_copy.at[index, "hakim"] = row["hakim"][0]

In [30]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

I Made Bagiarta~Hakim Ketua~Ya
Made Hermayanti Muliartha~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
Made Hermayanti Muliartha~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Ketua~Ya
Made Hermayanti Muliartha~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
Heriyanti~Hakim Ketua~Ya
I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
Made Hermayanti Muliartha~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Ketua~Ya
I Made Ba

In [31]:
for index, row in df_copy.iterrows():
    print(type(row['hakim']))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [32]:
def extract_hakim_ketua_name(hakim_string):
    if '~Hakim' in hakim_string:
        return hakim_string.split('~Hakim')[0].strip()
    return hakim_string 

In [33]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(extract_hakim_ketua_name)

In [34]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

I Made Bagiarta
Made Hermayanti Muliartha
I Made Bagiarta
I Gusti Made Juliartawan
I Made Bagiarta
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Ayu Kade Ari Wulandari
I Made Bagiarta
I Gusti Made Juliartawan
Made Hermayanti Muliartha
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Gusti Ayu Kade Ari Wulandari
Made Hermayanti Muliartha
I Made Bagiarta
I Gusti Made Juliartawan
I Made Bagiarta
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Made Juliartawan
Heriyanti
I Made Bagiarta
I Gusti Made Juliartawan
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Made Bagiarta
I Gusti Made Juliartawan
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Made Bagiarta
I Made Bagiarta
Made Hermayanti Muliartha
I Gusti Made Juliartawan
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Gusti Made Juliartawan
Heriyanti
I Gusti Made Juliartawan
Ni Made Kushandari
I Made Bagiarta
I Made Bagiarta
I Gusti Made Juliartawan
I Made Bagiarta
I Gusti Made Juliarta

In [35]:
df_copy

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,117/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",49 Hari,"['MADE SUDAMA ALIAS KARTOLO', 'PUTU WISNU ALIA...","['I MADE SUTAPA,S.H.']",Made Hermayanti Muliartha,3,['PUTU WISNU ALIAS WISNU~Subsider Penjara (10 ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudama ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,112/Pid.Sus/2024/PN Sgr,Narkotika,"Selasa, 30 Jul. 2024",44 Hari,"['I KOMANG TEKEN Alias TEKEN', 'KADEK ALBET SA...","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,4,['KADEK ALBET SANJAYA Alias ALBET~Pidana Penja...,M E N G A D I L I :\nMenyatakan terda...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...
3,Minutasi,101/Pid.B/2024/PN Sgr,Pencurian,"Kamis, 11 Jul. 2024",56 Hari,['ANDRIANI'],"['Nyoman Arif Budiman, S.H., M.H.']",I Gusti Made Juliartawan,3,['ANDRIANI~Pidana Penjara Waktu Tertentu (1 Ta...,"MENGADILI:\nMenyatakan Terdakwa ANDRIANI, tela...",“Demi Keadilan dan Kebenaran\nBerdasarkan Ketu...
4,Minutasi,98/Pid.Sus/2024/PN Sgr,Narkotika,"Rabu, 10 Jul. 2024",64 Hari,['I NYOMAN IWAN MAHADI alias MANG IWAN'],"['I GUSTI NGURAH ARYA DIATMIKA,S.H.']",I Made Bagiarta,3,['I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana ...,MENGADILI:\nMenyatakan Terdakwa I Nyoman Iwan ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
...,...,...,...,...,...,...,...,...,...,...,...,...
1162,Minutasi,237/Pid.B/2017/PN Sgr,Penggelapan,"Rabu, 20 Des. 2017",56 Hari,[Ayu Putu Erlyandani],"[IMAM EKA SETYAWAN, SH.]",Ni Made Dewi Sukrani,4,[Ayu Putu Erlyandani~Pidana Penjara Waktu Tert...,M E N G A D I L I:\nMenyatakan Terdakwa AYU PU...,Kesatu : Â \nPrimair.\n-------- Bahwa terdakwa...
1163,Minutasi,236/Pid.B/2017/PN Sgr,Kejahatan terhadap Nyawa,"Rabu, 20 Des. 2017",97 Hari,[Ketut Mahardika Alias Kelet],"[I Nyoman Sulitra, SH.,MH.]",Ni Made Dewi Sukrani,10,[Ketut Mahardika Alias Kelet~Pidana Penjara Wa...,MENGADILI:\nÂ \nMenyatakan Terdakwa Ketut Maha...,Kesatu\n----- Bahwa terdakwa Â KETUT MAHARDIKA...
1164,Minutasi,227/Pid.B/2017/PN Sgr,Penggelapan,"Senin, 11 Des. 2017",52 Hari,[Iwan Hermato Alias Iwan],"[I GEDE PUTU ASTAWA, SH.]",Ida Bagus Bama Dewa. P,3,[Iwan Hermato Alias Iwan~Pidana Kurungan (1 Ta...,MENGADILI\nMenyatakan Terdakwa Iwan Hermanto A...,Pertama : Pasal 372 Kitab Undang-undang Hukum ...
1165,Minutasi,226/Pid.Sus/2017/PN Sgr,Narkotika,"Senin, 11 Des. 2017",31 Hari,[Komang Irwin Pranata Alias Erwin],"[I GEDE PUTU ASTAWA, SH.]",Sudar,2,[Komang Irwin Pranata Alias Erwin~Pidana Penja...,Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...,Pertama : Pasal 112 ayat (1) UU RI No. 35 tahu...


### Data Cleaning: Barang Bukti

In [36]:
# for index, row in df_copy.iterrows():
#     print(f"Index: {index}, Terdakwa: {row['barang_bukti']}")

text = df_copy.iloc[-1]['barang_bukti']
print(text)

M E N G A D I L I
Â 
Menyatakan Terdakwa Jumat Ariyanto dengan identitas sebagaimana tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana â€œdengan sengaja mengangkut hasil hutan kayu yangÂ  tidak dilengkapi bersama dengan surat keterangan sahnya hasil hutanâ€;
Menjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selamaÂ  1 (satu) Â tahun dan denda sebesar Rp.500.000.000 (lima ratus juta rupiah) dengan ketentuan apabila denda tersebut tidak dibayar, maka akan diganti dengan pidana penjara pengganti selama 3 (tiga) bulan;
Â Menetapkan lamanya Terdakwa berada dalam penahanan dikurangkan seluruhnya dari pidana yang dijatuhkan ;
Menetapkan Terdakwa tetap berada dalam tahanan;
Menetapkan barang bukti berupa:
1 (satu) 1 (satu) batang kayu balang-balang bentuk bulan ukuran panjang 3 meter berdiameter 18 cm dengan volume 0,073 M3 ;
1 (satu) batang kayu balang-bala

In [37]:
# cleaned_text = re.sub(r'[^a-zA-Z0-9., /\\()\"\'-]+', '', text)
cleaned_text = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text)

In [38]:
print(cleaned_text)

M E N G A D I L I
 
Menyatakan Terdakwa Jumat Ariyanto dengan identitas sebagaimana tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana dengan sengaja mengangkut hasil hutan kayu yang  tidak dilengkapi bersama dengan surat keterangan sahnya hasil hutan
Menjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama  1 (satu)  tahun dan denda sebesar Rp.500.000.000 (lima ratus juta rupiah) dengan ketentuan apabila denda tersebut tidak dibayar, maka akan diganti dengan pidana penjara pengganti selama 3 (tiga) bulan
 Menetapkan lamanya Terdakwa berada dalam penahanan dikurangkan seluruhnya dari pidana yang dijatuhkan 
Menetapkan Terdakwa tetap berada dalam tahanan
Menetapkan barang bukti berupa
1 (satu) 1 (satu) batang kayu balang-balang bentuk bulan ukuran panjang 3 meter berdiameter 18 cm dengan volume 0,073 M3 
1 (satu) batang kayu balang-balang bentuk balok u

In [39]:
cleaned_text

'M E N G A D I L I\n \nMenyatakan Terdakwa Jumat Ariyanto dengan identitas sebagaimana tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana dengan sengaja mengangkut hasil hutan kayu yang  tidak dilengkapi bersama dengan surat keterangan sahnya hasil hutan\nMenjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama  1 (satu)  tahun dan denda sebesar Rp.500.000.000 (lima ratus juta rupiah) dengan ketentuan apabila denda tersebut tidak dibayar, maka akan diganti dengan pidana penjara pengganti selama 3 (tiga) bulan\n Menetapkan lamanya Terdakwa berada dalam penahanan dikurangkan seluruhnya dari pidana yang dijatuhkan \nMenetapkan Terdakwa tetap berada dalam tahanan\nMenetapkan barang bukti berupa\n1 (satu) 1 (satu) batang kayu balang-balang bentuk bulan ukuran panjang 3 meter berdiameter 18 cm dengan volume 0,073 M3 \n1 (satu) batang kayu balang-balang bentu

In [40]:
cleaned_text = re.sub(r' +', ' ', cleaned_text)
# cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
# cleaned_text = cleaned_text.strip()

In [41]:
print(cleaned_text)

M E N G A D I L I
 
Menyatakan Terdakwa Jumat Ariyanto dengan identitas sebagaimana tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana dengan sengaja mengangkut hasil hutan kayu yang tidak dilengkapi bersama dengan surat keterangan sahnya hasil hutan
Menjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama 1 (satu) tahun dan denda sebesar Rp.500.000.000 (lima ratus juta rupiah) dengan ketentuan apabila denda tersebut tidak dibayar, maka akan diganti dengan pidana penjara pengganti selama 3 (tiga) bulan
 Menetapkan lamanya Terdakwa berada dalam penahanan dikurangkan seluruhnya dari pidana yang dijatuhkan 
Menetapkan Terdakwa tetap berada dalam tahanan
Menetapkan barang bukti berupa
1 (satu) 1 (satu) batang kayu balang-balang bentuk bulan ukuran panjang 3 meter berdiameter 18 cm dengan volume 0,073 M3 
1 (satu) batang kayu balang-balang bentuk balok ukur

In [42]:
pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n\s*[A-Z]|$)"
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n[A-Z])"
match = re.search(pattern, cleaned_text, re.DOTALL)

if match:
    print(match.group().strip())
else:
    print("Tidak ada kecocokan ditemukan.")

Menetapkan barang bukti berupa
1 (satu) 1 (satu) batang kayu balang-balang bentuk bulan ukuran panjang 3 meter berdiameter 18 cm dengan volume 0,073 M3 
1 (satu) batang kayu balang-balang bentuk balok ukuran panjang 3 meter lebar 21 cm, tinggi 14 cm dengan volume 0,0882 M3


In [43]:
text = df_copy.iloc[3]['barang_bukti']
print(text)

MENGADILI:
Menyatakan Terdakwa ANDRIANI, telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana “Pencurian” sebagaimana dalam dakwaan tunggal Penuntut Umum;
Menjatuhkan pidana kepada Terdakwa, oleh karena itu dengan pidana penjara selama 1 ( satu ) tahun dan 2 ( dua ) bulan;
Menetapkan masa penangkapan dan penahanan yang telah dijalani oleh Terdakwa dikurangkan seluruhnya dari pidana penjara yang dijatuhkan;
Menetapkan Terdakwa tetap ditahan ;
Menetapkan barang bukti berupa:
1 (satu) buah gelang emas (batu ungu) berat 5 gram;
1 (satu) buah gelang emas bangle berat 3,5 gram;
1 (satu) buah gelang rantai emas berat 9,8 gram;
1 (satu) buah cincin emas mata Batu merah berat 2,5 gram;
2 (dua) pasang anting-anting emas;
1 (satu) buah dompet warna merah muda dengan motif  kupu-kupu;
1 (satu) lembar nota dari toko perhiasan emas “Dewata Ayu” tanggal 10 Oktober 2022 dengan nota No. KP 185.
1 (satu) kalung rantai ditaksir perhiasan emas 9 karat berat 12,2 / 12.0 Gram;
1 (satu) 

In [44]:
patterns = [
    r"barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"Barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"Barang\s*Bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"Barang\s*Bukti\s*Berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*Bukti\s*Berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*bukti\s*Berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*bukti,\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang-bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)"
]

for index, row in df_copy.iterrows():
    text_barang_bukti = row['barang_bukti']
    
    cleaned_barang_bukti = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text_barang_bukti)
    
    match_found = False

    for pattern in patterns:
        match = re.search(pattern, cleaned_barang_bukti, re.DOTALL)

        if match:
            df_copy.loc[index, 'cleaned_barang_bukti'] = match.group(1).strip()
            match_found = True
            break
    
    if not match_found:
        print(f"{index}. Tidak ada kecocokan ditemukan. {row['nomor_perkara']}")
        print(cleaned_barang_bukti)


40. Tidak ada kecocokan ditemukan. 68/Pid.B/2024/PN Sgr
MENGADILI
Menyatakan Terdakwa I Nyoman Eryanto Alias Munuk telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana penganiayaan sebagaimana dalam dakwaan tunggal
Menjatuhkan pidana terhadap Terdakwa dengan pidana penjara selama 4  (empat) bulan       
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan
Menetapkan agar Terdakwa tetap ditahan
Membebankan kepada Terdakwa untuk membayar biaya perkara sebesar Rp. 5.000. (lima ribu rupiah)
100. Tidak ada kecocokan ditemukan. 149/Pid.B/2023/PN Sgr
MENGADILI
Menyatakan Terdakwa PUTU HERRY SUARTHANA Alias HERIK sebagaimana identitas tersebut di atas, telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana penganiayaan sebagaimana dalam dakwaan tunggal  Penuntut Umum 
Menjatuhkan pidana terhadap Terdakwa oleh karena itu dengan pidana penjara selama 1 ( satu ) tahun
Menetapkan lamanya ma

In [45]:
df_copy

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
0,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,38 (tiga puluh delapan) potongan pipet plastik...
1,Minutasi,117/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",49 Hari,"['MADE SUDAMA ALIAS KARTOLO', 'PUTU WISNU ALIA...","['I MADE SUTAPA,S.H.']",Made Hermayanti Muliartha,3,['PUTU WISNU ALIAS WISNU~Subsider Penjara (10 ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudama ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik klip bening berisi butir...
2,Minutasi,112/Pid.Sus/2024/PN Sgr,Narkotika,"Selasa, 30 Jul. 2024",44 Hari,"['I KOMANG TEKEN Alias TEKEN', 'KADEK ALBET SA...","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,4,['KADEK ALBET SANJAYA Alias ALBET~Pidana Penja...,M E N G A D I L I :\nMenyatakan terda...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...,1.\n1 (satu) buah buah pipet kaca berisi resid...
3,Minutasi,101/Pid.B/2024/PN Sgr,Pencurian,"Kamis, 11 Jul. 2024",56 Hari,['ANDRIANI'],"['Nyoman Arif Budiman, S.H., M.H.']",I Gusti Made Juliartawan,3,['ANDRIANI~Pidana Penjara Waktu Tertentu (1 Ta...,"MENGADILI:\nMenyatakan Terdakwa ANDRIANI, tela...",“Demi Keadilan dan Kebenaran\nBerdasarkan Ketu...,1 (satu) buah gelang emas (batu ungu) berat 5 ...
4,Minutasi,98/Pid.Sus/2024/PN Sgr,Narkotika,"Rabu, 10 Jul. 2024",64 Hari,['I NYOMAN IWAN MAHADI alias MANG IWAN'],"['I GUSTI NGURAH ARYA DIATMIKA,S.H.']",I Made Bagiarta,3,['I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana ...,MENGADILI:\nMenyatakan Terdakwa I Nyoman Iwan ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah bong alat hisap sabu\n1 (satu) p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,Minutasi,237/Pid.B/2017/PN Sgr,Penggelapan,"Rabu, 20 Des. 2017",56 Hari,[Ayu Putu Erlyandani],"[IMAM EKA SETYAWAN, SH.]",Ni Made Dewi Sukrani,4,[Ayu Putu Erlyandani~Pidana Penjara Waktu Tert...,M E N G A D I L I:\nMenyatakan Terdakwa AYU PU...,Kesatu : Â \nPrimair.\n-------- Bahwa terdakwa...,1 (satu) buah Buku Simpanan atau Tabungan Kopr...
1163,Minutasi,236/Pid.B/2017/PN Sgr,Kejahatan terhadap Nyawa,"Rabu, 20 Des. 2017",97 Hari,[Ketut Mahardika Alias Kelet],"[I Nyoman Sulitra, SH.,MH.]",Ni Made Dewi Sukrani,10,[Ketut Mahardika Alias Kelet~Pidana Penjara Wa...,MENGADILI:\nÂ \nMenyatakan Terdakwa Ketut Maha...,Kesatu\n----- Bahwa terdakwa Â KETUT MAHARDIKA...,"1 (satu) buah pedang dengan panjang 55 cm, den..."
1164,Minutasi,227/Pid.B/2017/PN Sgr,Penggelapan,"Senin, 11 Des. 2017",52 Hari,[Iwan Hermato Alias Iwan],"[I GEDE PUTU ASTAWA, SH.]",Ida Bagus Bama Dewa. P,3,[Iwan Hermato Alias Iwan~Pidana Kurungan (1 Ta...,MENGADILI\nMenyatakan Terdakwa Iwan Hermanto A...,Pertama : Pasal 372 Kitab Undang-undang Hukum ...,1(satu) buah tas Kalep warna merah merk Son Al...
1165,Minutasi,226/Pid.Sus/2017/PN Sgr,Narkotika,"Senin, 11 Des. 2017",31 Hari,[Komang Irwin Pranata Alias Erwin],"[I GEDE PUTU ASTAWA, SH.]",Sudar,2,[Komang Irwin Pranata Alias Erwin~Pidana Penja...,Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...,Pertama : Pasal 112 ayat (1) UU RI No. 35 tahu...,1(satu) bungkus rokok merk Sampoerna Mild Men...


In [46]:
null_counts = df_copy.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara           0
nomor_perkara            0
klasifikasi_perkara      0
tanggal_pendaftaran      0
lama_proses              0
terdakwa                 0
penuntut_umum            0
hakim                    0
jumlah_saksi             0
putusan_hukuman          0
barang_bukti             0
dakwaan                  0
cleaned_barang_bukti    60
dtype: int64


In [47]:
df_copy = df_copy.dropna()

null_counts = df_copy.isnull().sum()
print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
dtype: int64


### Data Cleaning: Dakwaan

In [48]:
dakwaan = df_copy.iloc[1]['dakwaan']
print(dakwaan)

KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI BULELENG
Jl. Dewi Sartika No. 23, Kaliuntu, Kec. Buleleng, Kab. Buleleng, Bali
Telp. (0362) 22580. www.kejari-buleleng.go.id
“Demi Keadilan Dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa”
P-29
          SURAT DAKWAAN
NOMOR : REG. PERKARA PDM-47/Enz.2/BLL/07/2024
  IDENTITAS PARA TERDAKWA
Terdakwa I
Nama lengkap                :    MADE SUDAMA Alias KARTOLO
Nomor Identitas             :    5108073112800028
Tempat lahir                  :    Sangsit
Umur/tanggal lahir         :    43 Tahun / 31 Desember 1980
Jenis kelamin                 :    Laki-laki
Kewarganegaraan          :    Indonesia
Tempat tinggal               :    Banjar Dinas Beji, Desa Sangsit, Kecamatan Sawan, Kabupaten Buleleng
Agama                           :    Hindu
Pekerjaan                       :    Buruh Harian Lepas
Pendidikan                     :    SMA (Tamat)
  Terdakwa II
Nama lengkap                :    PUTU WISNU Alias WISNU
Nomor Identi

In [49]:
count_short_texts = sum(1 for text in df_copy['dakwaan'] if len(text) < 1000)
print(f"Jumlah teks dengan panjang di bawah 300 karakter: {count_short_texts}")

Jumlah teks dengan panjang di bawah 300 karakter: 33


In [50]:
# cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n]+', dakwaan)

# cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)

# print(cleaned_dakwaan)

cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', dakwaan)

cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)

print(cleaned_dakwaan)


KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI BULELENG
Jl. Dewi Sartika No. 23, Kaliuntu, Kec. Buleleng, Kab. Buleleng, Bali
Telp. (0362) 22580. www.kejari-buleleng.go.id
Demi Keadilan Dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa
P-29
          SURAT DAKWAAN
NOMOR  REG. PERKARA PDM-47/Enz.2/BLL/07/2024
  IDENTITAS PARA TERDAKWA
Terdakwa I
Nama lengkap                    MADE SUDAMA Alias KARTOLO
Nomor Identitas                 5108073112800028
Tempat lahir                      Sangsit
Umur/tanggal lahir             43 Tahun / 31 Desember 1980
Jenis kelamin                     Laki-laki
Kewarganegaraan              Indonesia
Tempat tinggal                   Banjar Dinas Beji, Desa Sangsit, Kecamatan Sawan, Kabupaten Buleleng
Agama                               Hindu
Pekerjaan                           Buruh Harian Lepas
Pendidikan                         SMA (Tamat)
  Terdakwa II
Nama lengkap                    PUTU WISNU Alias WISNU
Nomor Identitas           

In [51]:
for index, row in df_copy.iterrows():
    text_dakwaan = row['dakwaan']
    cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', text_dakwaan)

    cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
    cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
    df_copy.loc[index, 'cleaned_dakwaan'] = cleaned_dakwaan

C:\Users\Kalea\AppData\Local\Temp\ipykernel_28532\1074253341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy.loc[index, 'cleaned_dakwaan'] = cleaned_dakwaan


In [52]:
print(df_copy.iloc[0]["cleaned_dakwaan"])

KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI BULELENG
JL. Dewi Sartika Selatan No. 23 Singaraja  Bali 81116
Telp. (0362) 22580 www.kejari-buleleng.go.id
"Demi Keadilan dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa"
P-29
        SURAT DAKWAAN
NOMOR REG PERK  PDM   44 /Enz.2/BLL/07/2024
  A.     IDENTITAS TERDAKWA  
Nama Lengkap                 I NYOMAN SUARTA
Nomor Identitas                 5103051109720003
Tempat lahir                      Wanagiri
Umur/Tgl lahir                   51 Tahun / 11 September 1972
Jenis Kelamin                    Laki-laki      
Kebangsaan                      Indonesia
Tempat tinggal                   Banjar Dinas Yeh Ketipat, Desa Wanagiri, Kecamatan Sukasada, Kabupaten Buleleng
Agama                               Hindu
Pekerjaan                          Wiraswasta
Pendidikan                        S1
  B.     STATUS PENANGKAPAN DAN PENAHANAN    
1      Penangkapan                
Penangkapan                              Tanggal

In [53]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.findall(pattern, text_dakwaan)

    if matches: 
        last_match = matches[-1]
        df_copy.loc[index, "cleaned_dakwaan"] = last_match.strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"No dakwaan found. {index}")


No dakwaan found. 2
No dakwaan found. 22
No dakwaan found. 75
No dakwaan found. 81
No dakwaan found. 82
No dakwaan found. 85
No dakwaan found. 86
No dakwaan found. 87
No dakwaan found. 88
No dakwaan found. 97
No dakwaan found. 98
No dakwaan found. 99
No dakwaan found. 101
No dakwaan found. 102
No dakwaan found. 103
No dakwaan found. 105
No dakwaan found. 106
No dakwaan found. 107
No dakwaan found. 108
No dakwaan found. 109
No dakwaan found. 110
No dakwaan found. 112
No dakwaan found. 113
No dakwaan found. 114
No dakwaan found. 115
No dakwaan found. 116
No dakwaan found. 117
No dakwaan found. 118
No dakwaan found. 119
No dakwaan found. 120
No dakwaan found. 121
No dakwaan found. 122
No dakwaan found. 123
No dakwaan found. 124
No dakwaan found. 125
No dakwaan found. 126
No dakwaan found. 127
No dakwaan found. 128
No dakwaan found. 129
No dakwaan found. 131
No dakwaan found. 132
No dakwaan found. 133
No dakwaan found. 134
No dakwaan found. 137
No dakwaan found. 139
No dakwaan found. 141
N

In [54]:
df_copy

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...
1,Minutasi,117/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",49 Hari,"['MADE SUDAMA ALIAS KARTOLO', 'PUTU WISNU ALIA...","['I MADE SUTAPA,S.H.']",Made Hermayanti Muliartha,3,['PUTU WISNU ALIAS WISNU~Subsider Penjara (10 ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudama ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik klip bening berisi butir...,DAKWAAN \nKESATU\n Bahwa ia Terdakw...
2,Minutasi,112/Pid.Sus/2024/PN Sgr,Narkotika,"Selasa, 30 Jul. 2024",44 Hari,"['I KOMANG TEKEN Alias TEKEN', 'KADEK ALBET SA...","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,4,['KADEK ALBET SANJAYA Alias ALBET~Pidana Penja...,M E N G A D I L I :\nMenyatakan terda...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...,1.\n1 (satu) buah buah pipet kaca berisi resid...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...
3,Minutasi,101/Pid.B/2024/PN Sgr,Pencurian,"Kamis, 11 Jul. 2024",56 Hari,['ANDRIANI'],"['Nyoman Arif Budiman, S.H., M.H.']",I Gusti Made Juliartawan,3,['ANDRIANI~Pidana Penjara Waktu Tertentu (1 Ta...,"MENGADILI:\nMenyatakan Terdakwa ANDRIANI, tela...",“Demi Keadilan dan Kebenaran\nBerdasarkan Ketu...,1 (satu) buah gelang emas (batu ungu) berat 5 ...,DAKWAAN \nBahwa ia TERDAKWA ANDRIANI selanjutn...
4,Minutasi,98/Pid.Sus/2024/PN Sgr,Narkotika,"Rabu, 10 Jul. 2024",64 Hari,['I NYOMAN IWAN MAHADI alias MANG IWAN'],"['I GUSTI NGURAH ARYA DIATMIKA,S.H.']",I Made Bagiarta,3,['I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana ...,MENGADILI:\nMenyatakan Terdakwa I Nyoman Iwan ...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah bong alat hisap sabu\n1 (satu) p...,DAKWAAN \nKESATU\n-Bahwa Terdakwa I Nyoman Iw...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,Minutasi,237/Pid.B/2017/PN Sgr,Penggelapan,"Rabu, 20 Des. 2017",56 Hari,[Ayu Putu Erlyandani],"[IMAM EKA SETYAWAN, SH.]",Ni Made Dewi Sukrani,4,[Ayu Putu Erlyandani~Pidana Penjara Waktu Tert...,M E N G A D I L I:\nMenyatakan Terdakwa AYU PU...,Kesatu : Â \nPrimair.\n-------- Bahwa terdakwa...,1 (satu) buah Buku Simpanan atau Tabungan Kopr...,Kesatu \nPrimair.\n- Bahwa terdakwa AYU PUTU...
1163,Minutasi,236/Pid.B/2017/PN Sgr,Kejahatan terhadap Nyawa,"Rabu, 20 Des. 2017",97 Hari,[Ketut Mahardika Alias Kelet],"[I Nyoman Sulitra, SH.,MH.]",Ni Made Dewi Sukrani,10,[Ketut Mahardika Alias Kelet~Pidana Penjara Wa...,MENGADILI:\nÂ \nMenyatakan Terdakwa Ketut Maha...,Kesatu\n----- Bahwa terdakwa Â KETUT MAHARDIKA...,"1 (satu) buah pedang dengan panjang 55 cm, den...",Kesatu\n- Bahwa terdakwa KETUT MAHARDIKA Als ...
1164,Minutasi,227/Pid.B/2017/PN Sgr,Penggelapan,"Senin, 11 Des. 2017",52 Hari,[Iwan Hermato Alias Iwan],"[I GEDE PUTU ASTAWA, SH.]",Ida Bagus Bama Dewa. P,3,[Iwan Hermato Alias Iwan~Pidana Kurungan (1 Ta...,MENGADILI\nMenyatakan Terdakwa Iwan Hermanto A...,Pertama : Pasal 372 Kitab Undang-undang Hukum ...,1(satu) buah tas Kalep warna merah merk Son Al...,Pertama Pasal 372 Kitab Undang-undang Hukum P...
1165,Minutasi,226/Pid.Sus/2017/PN Sgr,Narkotika,"Senin, 11 Des. 2017",31 Hari,[Komang Irwin Pranata Alias Erwin],"[I GEDE PUTU ASTAWA, SH.]",Sudar,2,[Komang Irwin Pranata Alias Erwin~Pidana Penja...,Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â Â ...,Pertama : Pasal 112 ayat (1) UU RI No. 35 tahu...,1(satu) bungkus rokok merk Sampoerna Mild Men...,Pertama Pasal 112 ayat (1) UU RI No. 35 tahun...


In [55]:
print(df_copy.iloc[4]['cleaned_dakwaan'])


DAKWAAN  
KESATU
-Bahwa Terdakwa I Nyoman Iwan Mahadi Alias Mang Iwan, pada hari Jumat, tanggal 22 Maret 2024, sekitar jam 11.15 Wita atau setidak-tidaknya pada suatu waktu lain dalam bulan Maret 2024, bertempat di Banjar Dinas Bukit Sakti, Desa Lokapaksa, Kecamatan Seririt, Kabupaten Buleleng atau setidak-tidaknya pada suatu tempat lain yang termasuk Daerah Hukum Pengadilan Negeri Singaraja, percobaan atau permufakatan jahat untuk melakukan tindak pidana narkotika dan prekursor narkotika yakni secara tanpa hak atau melawan hukum memiliki, menyimpan, menguasai, atau menyediakan Narkotika Golongan I bukan tanaman, dilakukan Terdakwa dengan cara-cara sebagai berikut  -
Berawal  pada hari Jumat, tanggal 22 Maret 2024, sekitar jam 11.05 Wita, saksi I Made Sudiastika, SE bersama  sama dengan saksi Ketut Agus Suryada Sukahat, yang keduanya adalah anggota Sat Narkoba Polres Buleleng, mendapat informasi dari masyarakat bahwa di Desa Lokapaksa sering terjadi Penyalahgunaan Narkotika dan berdasa

In [56]:
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

## Data Transformation

### Data Transformation: Copy to New DataFrame

In [57]:
df_copy.head(1)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...


In [78]:
df_clean = pd.DataFrame()

selected_columns = ['nomor_perkara', 'klasifikasi_perkara', 'terdakwa', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'putusan_hukuman', 'cleaned_barang_bukti', 'cleaned_dakwaan']
df_clean = df_copy[selected_columns].copy()

df_clean.head(1)

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,118/Pid.Sus/2024/PN Sgr,Narkotika,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...


### Data Transformation: Extract Multiple Terdakwa & Putusan Hukuman to each Row 

In [79]:
print(type(df_clean.iloc[1]['terdakwa']))

<class 'str'>


In [80]:
def clean_and_convert_to_list(terdakwa):
    # Ensure the input is treated as a string
    if isinstance(terdakwa, list):
        # If it's already a list, return it as is
        return terdakwa
    if isinstance(terdakwa, str):
        # Clean and convert only if it's a string
        cleaned_str = re.sub(r'\s+', ' ', terdakwa.strip())  # Clean whitespace
        try:
            # Use literal_eval to convert if it's a proper list
            result_list = ast.literal_eval(cleaned_str)
        except (ValueError, SyntaxError):
            # If there's an error, just return as a single-element list
            result_list = [cleaned_str]

        # Ensure the result is a list
        if isinstance(result_list, str):
            return [result_list]
        return result_list
    return []


# Apply the cleaning and conversion function
df_clean['terdakwa'] = df_clean['terdakwa'].apply(clean_and_convert_to_list)
df_clean['putusan_hukuman'] = df_clean['putusan_hukuman'].apply(clean_and_convert_to_list)

In [81]:
print(type(df_clean.iloc[1]['terdakwa']))
print(type(df_clean.iloc[1]['putusan_hukuman']))

df_clean.head()

<class 'list'>
<class 'list'>


,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,118/Pid.Sus/2024/PN Sgr,Narkotika,[I NYOMAN SUARTA],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,[I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\n...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...
1,117/Pid.Sus/2024/PN Sgr,Narkotika,"[MADE SUDAMA ALIAS KARTOLO, PUTU WISNU ALIAS W...","['I MADE SUTAPA,S.H.']",Made Hermayanti Muliartha,3,[PUTU WISNU ALIAS WISNU~Subsider Penjara (10 B...,1 (satu) buah plastik klip bening berisi butir...,DAKWAAN \nKESATU\n Bahwa ia Terdakw...
2,112/Pid.Sus/2024/PN Sgr,Narkotika,"[I KOMANG TEKEN Alias TEKEN, KADEK ALBET SANJA...","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,4,[KADEK ALBET SANJAYA Alias ALBET~Pidana Penjar...,1.\n1 (satu) buah buah pipet kaca berisi resid...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...
3,101/Pid.B/2024/PN Sgr,Pencurian,[ANDRIANI],"['Nyoman Arif Budiman, S.H., M.H.']",I Gusti Made Juliartawan,3,[ANDRIANI~Pidana Penjara Waktu Tertentu (1 Tah...,1 (satu) buah gelang emas (batu ungu) berat 5 ...,DAKWAAN \nBahwa ia TERDAKWA ANDRIANI selanjutn...
4,98/Pid.Sus/2024/PN Sgr,Narkotika,[I NYOMAN IWAN MAHADI alias MANG IWAN],"['I GUSTI NGURAH ARYA DIATMIKA,S.H.']",I Made Bagiarta,3,[I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana P...,1 (satu) buah bong alat hisap sabu\n1 (satu) p...,DAKWAAN \nKESATU\n-Bahwa Terdakwa I Nyoman Iw...


In [100]:
# Find rows where the lengths of "terdakwa" and "putusan_hukuman" are different
mismatched_rows = df_clean[df_clean['terdakwa'].str.len() != df_clean['putusan_hukuman'].str.len()]

# Display rows with mismatched counts
print("Rows with mismatched element counts:")
print(mismatched_rows)

df_clean_temp = df_clean[df_clean['terdakwa'].str.len() == df_clean['putusan_hukuman'].str.len()]

# Reset the index after dropping the rows (optional)
df_clean_temp = df_clean_temp.reset_index(drop=True)



Rows with mismatched element counts:
                 nomor_perkara          klasifikasi_perkara  \
18   93/Pid.Sus-LH/2024/PN Sgr  Konservasi Sumber Daya Alam   
414      147/Pid.B/2021/PN Sgr                    Pencurian   

                                              terdakwa  \
18   [KETUT SUMANTRA alias LOTOT, MOCH HASAN BASRI ...   
414  [IDA BAGUS ARI WIBAWA Als. GUS UNENG, KKADEK D...   

                    penuntut_umum                              hakim  \
18   ['Kadek Adi Pramarta, S.H.']                    I Made Bagiarta   
414     [Ida Kade Widiatmika, SH]  Anak Agung Ngurah Budhi Dharmawan   

     jumlah_saksi                                    putusan_hukuman  \
18              4  [MOCH HASAN BASRI alias BAS~Pidana Denda Rp.5....   
414             4  [IDA BAGUS ARI WIBAWA Als. GUS UNENG~Pidana Pe...   

                                  cleaned_barang_bukti  \
18   1 (satu) pucuk senapan angin warna hitam rakit...   
414  1 (satu) buah laptop merk axioo warna hitam

In [93]:
print(df_clean.loc[18, "putusan_hukuman"])
print(df_clean.loc[18, "terdakwa"])


['MOCH HASAN BASRI alias BAS~Pidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )\nSubsider Kurungan (3 Bulan )', 'KETUT SUMANTRA alias LOTOT~Subsider Kurungan (3 Bulan )\nPidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )', 'MOCH HASAN BASRI alias BAS~Pidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )\nSubsider Kurungan (3 Bulan )']
['KETUT SUMANTRA alias LOTOT', 'MOCH HASAN BASRI alias BAS']


In [101]:
# df_clean_terdakwa_exploded = df_clean.explode('terdakwa').reset_index(drop=True)
# df_clean_putusanhukuman_exploded = df_clean_terdakwa_exploded.explode('putusan_hukuman').reset_index(drop=True)

df_clean_terdakwa_exploded = df_clean_temp.explode(['terdakwa', 'putusan_hukuman']).reset_index(drop=True)

# Display the result
print(len(df_clean_temp))
print(len(df_clean_terdakwa_exploded))

df_clean_terdakwa_exploded.head()

1105
1276


,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,118/Pid.Sus/2024/PN Sgr,Narkotika,I NYOMAN SUARTA,"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\nP...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...
1,117/Pid.Sus/2024/PN Sgr,Narkotika,MADE SUDAMA ALIAS KARTOLO,"['I MADE SUTAPA,S.H.']",Made Hermayanti Muliartha,3,PUTU WISNU ALIAS WISNU~Subsider Penjara (10 Bu...,1 (satu) buah plastik klip bening berisi butir...,DAKWAAN \nKESATU\n Bahwa ia Terdakw...
2,117/Pid.Sus/2024/PN Sgr,Narkotika,PUTU WISNU ALIAS WISNU,"['I MADE SUTAPA,S.H.']",Made Hermayanti Muliartha,3,MADE SUDAMA ALIAS KARTOLO~Subsider Penjara (1 ...,1 (satu) buah plastik klip bening berisi butir...,DAKWAAN \nKESATU\n Bahwa ia Terdakw...
3,112/Pid.Sus/2024/PN Sgr,Narkotika,I KOMANG TEKEN Alias TEKEN,"['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,4,KADEK ALBET SANJAYA Alias ALBET~Pidana Penjara...,1.\n1 (satu) buah buah pipet kaca berisi resid...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...
4,112/Pid.Sus/2024/PN Sgr,Narkotika,KADEK ALBET SANJAYA Alias ALBET,"['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,4,I KOMANG TEKEN Alias TEKEN~Pidana Penjara Wakt...,1.\n1 (satu) buah buah pipet kaca berisi resid...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...


In [102]:
df_clean_terdakwa_exploded.iloc[1]['putusan_hukuman']

'PUTU WISNU ALIAS WISNU~Subsider Penjara (10 Bulan )'

In [105]:
type(df_clean_terdakwa_exploded.iloc[1]['putusan_hukuman'])

str

In [109]:
print(len(df_clean_terdakwa_exploded['terdakwa']))
print(len(df_clean_terdakwa_exploded['putusan_hukuman']))

1276
1276


In [111]:
for i in range(10):
    print(i, ".", df_clean_terdakwa_exploded.iloc[i]['putusan_hukuman'])

0 . I NYOMAN SUARTA~Subsider Penjara (2 Bulan )
Pidana Denda Rp.1.000.000.000,00
Pidana Penjara Waktu Tertentu (4 Tahun 4 Bulan )
1 . PUTU WISNU ALIAS WISNU~Subsider Penjara (10 Bulan )
2 . MADE SUDAMA ALIAS KARTOLO~Subsider Penjara (1 Tahun 2 Bulan )
3 . KADEK ALBET SANJAYA Alias ALBET~Pidana Penjara Waktu Tertentu (9 Bulan )
4 . I KOMANG TEKEN Alias TEKEN~Pidana Penjara Waktu Tertentu (9 Bulan )
5 . ANDRIANI~Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
6 . I NYOMAN IWAN MAHADI alias MANG IWAN~Pidana Penjara Waktu Tertentu (9 Bulan )
7 . KOMANG ADITYA~Pidana Penjara Waktu Tertentu (4 Bulan )
8 . KETUT SUBAGIA~Pidana Penjara Waktu Tertentu (4 Bulan )
9 . MUCH.ARIFIN~Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )


In [145]:
IXA = 90
print(df_clean_terdakwa_exploded.iloc[IXA]['putusan_hukuman'])
print(df_clean_terdakwa_exploded.iloc[IXA]['nomor_perkara'])

LUH SUKRANADI ASTARI~Subsider Kurungan (1 Bulan )
Pidana Denda Rp.5.000.000,00
Pidana Penjara Waktu Tertentu (1 Bulan 15 Hari)
15/Pid.Sus/2024/PN Sgr


In [151]:
pattern = r"Pidana Penjara Waktu Tertentu\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_2 = r"Subsider Penjara\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_3 = r"Subsider Kurungan\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"

for index, row in df_clean_terdakwa_exploded.iterrows():
    # Use re.search to find the match
    text = row["putusan_hukuman"]
    match = re.search(pattern, text, re.IGNORECASE)
    match_2 = re.search(pattern_2, text, re.IGNORECASE)
    match_3 = re.search(pattern_3, text, re.IGNORECASE)

    # Extract and print the result
    if match:
        extracted_duration = match.group(0)
        print(index,"-", extracted_duration)

    elif match_2:
        extracted_duration = match_2.group(0)
        print(index,"-", extracted_duration)

    elif match_3:
        extracted_duration = match_3.group(0)
        print(index,"-", extracted_duration)
        
    else:
        print(f"{index} - No match found.")

0 - Pidana Penjara Waktu Tertentu (4 Tahun 4 Bulan )
1 - Subsider Penjara (10 Bulan )
2 - Subsider Penjara (1 Tahun 2 Bulan )
3 - Pidana Penjara Waktu Tertentu (9 Bulan )
4 - Pidana Penjara Waktu Tertentu (9 Bulan )
5 - Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
6 - Pidana Penjara Waktu Tertentu (9 Bulan )
7 - Pidana Penjara Waktu Tertentu (4 Bulan )
8 - Pidana Penjara Waktu Tertentu (4 Bulan )
9 - Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )
10 - Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
11 - Pidana Penjara Waktu Tertentu (9 Bulan )
12 - Pidana Penjara Waktu Tertentu (9 Bulan )
13 - Pidana Penjara Waktu Tertentu (1 Tahun )
14 - Pidana Penjara Waktu Tertentu (9 Bulan )
15 - Pidana Penjara Waktu Tertentu (1 Tahun )
16 - Pidana Penjara Waktu Tertentu (1 Tahun 3 Bulan )
17 - Pidana Penjara Waktu Tertentu (6 Tahun )
18 - Pidana Penjara Waktu Tertentu (4 Tahun )
19 - Pidana Penjara Waktu Tertentu (4 Tahun )
20 - Pidana Penjara Waktu Tertentu (6 Bulan )
21 - Pidana Penjara W

## Data Validation

# Exploratory Data Analysis

# Model Preprocessing

## Data Normalization

## Text Summarization

In [149]:
import re

# Refined regex to be more robust with flexible spaces and handle trailing spaces around Tahun/Bulan
pattern_1 = r"Waktu Tertentu\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"  # Flexible space handling
pattern_2 = r"Subsider Penjara\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"  # For "Subsider Penjara"

# Example texts
texts = [
    "KADEK ALBET SANJAYA Alias ALBET~Pidana Penjara Waktu Tertentu (9 Bulan)",
    "KADEK ALBET SANJAYA Alias ALBET~Pidana Penjara Waktu Tertentu (9 Tahun)",
    "KADEK ALBET SANJAYA Alias ALBET~Pidana Penjara Subsider Penjara (9 Tahun 3 Bulan)",
    "KADEK ALBET SANJAYA Alias ALBET~Pidana Penjara Subsider Penjara (9 Bulan)",
    "FEBRI KRISWANTO~Pidana Penjara Waktu Tertentu (1 Tahun )",  # This case
    "GDE NGURAH DICKY KUSUMAWARDHANA Alias NGURAH~Pidana Penjara Waktu Tertentu (1 Tahun )",  # This case
    "febri kriswanto~pidana penjara waktu tertentu (1 tahun )",  # Case insensitivity test
    "KARYA NANDA~Pidana Penjara Waktu Tertentu (1 Tahun)",  # Single year case
]

# Applying the regular expressions to each text with re.IGNORECASE for case insensitivity
for text in texts:
    match_1 = re.search(pattern_1, text, re.IGNORECASE)  # Case-insensitive search
    match_2 = re.search(pattern_2, text, re.IGNORECASE)  # Case-insensitive search
    
    if match_1:
        print(f"Matched Waktu Tertentu: {match_1.group(0)}")
    elif match_2:
        print(f"Matched Subsider Penjara: {match_2.group(0)}")
    else:
        print("No match found.")


Matched Waktu Tertentu: Waktu Tertentu (9 Bulan)
Matched Waktu Tertentu: Waktu Tertentu (9 Tahun)
Matched Subsider Penjara: Subsider Penjara (9 Tahun 3 Bulan)
Matched Subsider Penjara: Subsider Penjara (9 Bulan)
Matched Waktu Tertentu: Waktu Tertentu (1 Tahun )
Matched Waktu Tertentu: Waktu Tertentu (1 Tahun )
Matched Waktu Tertentu: waktu tertentu (1 tahun )
Matched Waktu Tertentu: Waktu Tertentu (1 Tahun)
